In [1]:
from pybis import Openbis
import json
import ipywidgets as widgets
from IPython.display import display, Javascript

In [ ]:
# Global variables
OPENBIS_SESSION = None

In [ ]:
# Widgets
crystals_dropdown = widgets.Dropdown(description='Select a crystal:', disabled=False,)
crystals_dropdown.style = {'description_width': 'initial'}
crystal_details_textbox = widgets.Textarea(value = '', description = '', disabled = True, layout = widgets.Layout(width = '50%', height = '200px'))
sample_name_textbox = widgets.Text(value = '', placeholder = 'Enter sample name', description = 'Sample name:', disabled = False)
sample_name_textbox.style = {'description_width': 'initial'}
create_sample_button = widgets.Button(description = 'Create sample', disabled = False, button_style = '', tooltip = 'Create sample')
quit_button = widgets.Button(description = 'Go back', disabled = False, button_style = '', tooltip = 'Go back')

In [ ]:
# Functions
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Function to close the notebook page
def close_notebook(b):
    display(Javascript(f'window.open("home.ipynb", "_blank")'))
    display(Javascript('window.close()'))

def create_sample_action(b):
    global OPENBIS_SESSION
    
    if crystals_dropdown.value == "-1":
        sample_parents = []
    else:
        sample_parents = [crystals_dropdown.value]
    
    OPENBIS_SESSION.new_object(type = "SAMPLE", 
                            collection = "20240716082516377-60",
                            props = {"$name": sample_name_textbox.value},
                            parents = sample_parents).save()

def load_crystal_metadata(change):
    global OPENBIS_SESSION
    if crystals_dropdown.value == "-1":
        crystal_details_textbox.value = ""
    else:
        crystal_metadata = OPENBIS_SESSION.get_object(crystals_dropdown.value).props.all()
        property_list = [("Name", "$name"), ("Material", "material"), ("Face", "face"), ("Sample Plate", "sample_plate"), ("Diameter", "diameter"), ("Height", "height"), ("Specifications", "specifications")]
        crystal_metadata_string = ""
        for property in property_list:
            if property[1] in ["diameter", "height"]:
                if crystal_metadata[property[1]] is None:
                    crystal_metadata_string = f"{crystal_metadata_string} {property[0]}: {crystal_metadata[property[1]]}\n"
                else:
                    property_dict = json.loads(crystal_metadata[property[1]])
                    crystal_metadata_string = f"{crystal_metadata_string} {property[0]}: {property_dict['value']} {property_dict['unit']}\n"
            else:
                crystal_metadata_string = f"{crystal_metadata_string} {property[0]}: {crystal_metadata[property[1]]}\n"
        crystal_details_textbox.value = crystal_metadata_string

In [ ]:
# Connect with openBIS
session_data = read_json("token.json")
OPENBIS_SESSION = Openbis(session_data["url"], verify_certificates = False)
OPENBIS_SESSION.set_token(session_data["token"])

## Create a sample object in openBIS

In [ ]:
crystals = OPENBIS_SESSION.get_objects(type = "CRYSTAL")
crystals_names_permids = [(crystal.props["$name"], crystal.permId) for crystal in crystals]
crystals_names_permids.insert(0, ('No crystal', "-1"))
crystals_dropdown.options = crystals_names_permids

bottom_buttons_hbox = widgets.HBox([create_sample_button, quit_button])
crystal_metadata_boxes = widgets.HBox([crystals_dropdown, crystal_details_textbox])
display(crystal_metadata_boxes, sample_name_textbox, bottom_buttons_hbox)
create_sample_button.on_click(create_sample_action)
quit_button.on_click(close_notebook)
crystals_dropdown.observe(load_crystal_metadata, names = 'value')